# Homework
## AWQ 实战 - OPT-6.7B

**加载模型**

In [2]:
from awq import AutoAWQForCausalLM
from transformers import AutoTokenizer

model_name_or_path = "facebook/opt-6.7b"
quant_model_dir = "models/opt-6.7b-awq"

quant_config = {
    "zero_point": True,
    "q_group_size": 128,
    "w_bit": 4,
    "version": "GEMM"
}

# 加载模型
model = AutoAWQForCausalLM.from_pretrained(model_name_or_path, trust_remote_code=True)
tokenizer = AutoTokenizer.from_pretrained(model_name_or_path, trust_remote_code=True)

/root/miniconda3/envs/LLM/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]/root/miniconda3/envs/LLM/lib/python3.11/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
Loading checkpoint shards: 100%|██████████| 2/2 [00:01<00:00,  1.02it/s]


**量化**

In [3]:
# 量化模型
model.quantize(tokenizer, quant_config=quant_config)

/root/miniconda3/envs/LLM/lib/python3.11/site-packages/huggingface_hub/repocard.py:105: UserWarning: Repo card metadata block was not found. Setting CardData to empty.
  warnings.warn("Repo card metadata block was not found. Setting CardData to empty.")
AWQ: 100%|██████████| 32/32 [09:21<00:00, 17.55s/it]


貌似使用AWQ量化，显存占用要少很多

**Transfromers 兼容性配置**

In [4]:
from transformers import AwqConfig, AutoConfig

# 修改配置文件以使其与transformers集成兼容
quantization_config = AwqConfig(
    bits=quant_config["w_bit"],
    group_size=quant_config["q_group_size"],
    zero_point=quant_config["zero_point"],
    version=quant_config["version"].lower(),
).to_dict()

# 预训练的transformers模型存储在model属性中，我们需要传递一个字典
model.model.config.quantization_config = quantization_config

In [5]:
# 保存模型权重
model.save_quantized(quant_model_dir)
# 保存分词器
tokenizer.save_pretrained(quant_model_dir)  

('models/opt-6.7b-awq/tokenizer_config.json',
 'models/opt-6.7b-awq/special_tokens_map.json',
 'models/opt-6.7b-awq/vocab.json',
 'models/opt-6.7b-awq/merges.txt',
 'models/opt-6.7b-awq/added_tokens.json',
 'models/opt-6.7b-awq/tokenizer.json')

### 加载量化模型，并测试

In [6]:
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained(quant_model_dir)
model = AutoModelForCausalLM.from_pretrained(quant_model_dir, device_map="cuda").to(0)

In [7]:
def generate_text(text):
    inputs = tokenizer(text, return_tensors="pt").to(0)

    out = model.generate(**inputs, max_new_tokens=64)
    return tokenizer.decode(out[0], skip_special_tokens=True)

In [8]:
result = generate_text("Merry Christmas! I'm glad to")
print(result)

Merry Christmas! I'm glad to my family
 merry merry me! I'm merry! Merry Christmas!
Merry Christmas is me and my family!


In [9]:
result = generate_text("The woman worked as a")
print(result)

The woman worked as a caretaker.
Yeah a woman who worked as a caretaker.
Woman who worked as a caretaker, and they are all a woman who worked as a caretaker? I don't work, you don't work, you don't work.  I didn don work work, you don't work, you


**有效果，但是回答一言难尽，不太能用，主要原因还是模型本身**